In [1]:
# In this script, we will read in the surface air temp from CMIP5 output,
# calculate the change in air temperature. Then, we will plot this on a latitude-height plot

# By: Ty Janoski
# updated: 06.09.20

In [2]:
# Import libraries

import xarray as xr
import numpy as np
import xesmf as xe
import dask.array as da
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [ ]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']

# read in CAM5 kernel for lat/lon grid
ds_ker = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/'+
                         'CMIP5_Arctic_Amplification/'+
                         'CAM5_kernels/alb.kernel.nc',decode_times=False)
lats = ds_ker.lat
lons = ds_ker.lon

del ds_ker

# make empty dataset with lat/lon for regridding later on
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )

# some models have extra pressure levels, we only want standard pressure levels
# that match the CAM5 kernels


# this list will hold the abrupt4xCO2 - piControl air temperature
zonal = []
for mod in models:
    print(mod)
    
    # read in surface pressure for masking
    ps = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ps_Amon_'+
                                  mod+'_*.nc',combine='by_coords',decode_cf=True
                          ).ps.isel(time=slice(-600,None))
    
    # read in air temperature
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ta_Amon_'+
                                  mod+'_*.nc',combine='by_coords',decode_cf=True
                          ).ta.isel(time=slice(-600,None))
    
    # for some reason, ACCESS1-0 and ACCESS1-3 have slightly different grids
    # for 2D and 3D variables. We have to fix this :(
    if(mod in ['ACCESS1-0','ACCESS1-3'])    :
        access_regridder = xe.Regridder(ps,ta,
                                        'bilinear',periodic=True,reuse_weights=True)
        ps = access_regridder(ps.compute())
    
    
    # make regridder
    regridder = xe.Regridder(ps,ds_out,'bilinear',periodic=True,reuse_weights=True)
    
    
    # remove air temperatures below the surface
    ta = ta.where(ta.plev < ps)
    
    # lastly, make climatology
    clim = ta.groupby(ta.time.dt.month).mean(dim='time')
    
    # repeat process for abrupt4xCO2 runs
    ps = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ps_Amon_'+
                                  mod+'_*.nc',combine='by_coords',decode_cf=True
                          ).ps.isel(time=slice(None,1800))
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ta_Amon_'+
                                  mod+'_*.nc',combine='by_coords',decode_cf=True
                          ).ta.isel(time=slice(None,1800))
    
    # for some reason, ACCESS1-0 and ACCESS1-3 have slightly different grids
    # for 2D and 3D variables. We have to fix this :(
    if(mod in ['ACCESS1-0','ACCESS1-3']):
        ps = access_regridder(ps.compute())
        
    ta = ta.where(ta.plev < ps)
    
    # take difference between abrupt4xCO2 and piControl climatology
    diff = ta - da.tile(clim,(150,1,1,1))
    
    # regrid, take zonal mean, add to list
    zonal.append(regridder(diff.compute()).mean(dim='lon'))

# save output as netcdf file
for z in range(len(zonal[:])):
    zonal[z]['time'] = np.arange(0,1800,1)
    zonal[z] = zonal[z].rename('ta')
    zonal[z].to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+models[z]+
                       '_zonal_ta.nc')

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/*_zonal_ta.nc',
                        parallel=True, combine='nested',
                       concat_dim='model',use_cftime=True).__xarray_dataarray_variable__
plevs = xr.open_dataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/'+
                                'ta_Amon_ACCESS1-0_abrupt4xCO2_'+
                                'r1i1p1_030001-034912.nc',use_cftime=True).plev
ta = ta.sel(plev = plevs)

In [5]:
ta_mon = ta.isel(time=slice(0,3)).sel(lat=slice(0,None),plev=slice(None,30000)).mean(
    dim=['model','time']).compute()
ta_ann = ta.isel(time=slice(1200,1800)).sel(lat=slice(0,None),plev=slice(None,30000)).mean(
    dim=['model','time']).compute()
ta_year = ta.isel(time=slice(0,12)).sel(lat=slice(0,None),plev=slice(None,30000)).mean(
    dim=['model','time']).compute()

In [29]:
colors_neg = plt.cm.RdBu_r(np.linspace(0.466, 0.5, 256))
colors_pos = plt.cm.RdBu_r(np.linspace(0.5, 1, 256))
all_colors = np.vstack((colors_neg,colors_pos))
orig = colors.LinearSegmentedColormap.from_list('orig',
    all_colors)
divnorm = colors.DivergingNorm(vmin=-1, vcenter=0, vmax=1.9)

fig, axes = plt.subplots(ncols=3,figsize=(7.4,3),sharey=True,sharex=True)

# plot away!
im1 = axes[0].contourf(ta_mon.lat,ta_mon.plev/100,ta_mon,norm=divnorm,
                       cmap=orig,levels=np.arange(-0.2,1.9,0.2))
axes[0].invert_yaxis()
cb1 = plt.colorbar(im1,ax=axes[0])
cb1.ax.tick_params(labelsize=8)
cb1.set_ticks(np.arange(-0.2,1.9,0.2))

##########################################################

colors_neg = plt.cm.RdBu_r(np.linspace(0.466, 0.5, 256))
colors_pos = plt.cm.RdBu_r(np.linspace(0.5, 1, 256))
all_colors = np.vstack((colors_neg,colors_pos))
orig = colors.LinearSegmentedColormap.from_list('orig',
    all_colors)
divnorm = colors.DivergingNorm(vmin=-1, vcenter=0, vmax=2.6)


im2 = axes[1].contourf(ta_year.lat, ta_year.plev/100, ta_year,
                      cmap=orig,norm=divnorm,levels=np.arange(0,2.7,0.2))
cb2 = plt.colorbar(im2,ax=axes[1])
cb2.ax.tick_params(labelsize=8)
cb2.set_ticks(np.arange(0,2.7,0.2))

###########################################################

colors_neg = plt.cm.RdBu_r(np.linspace(0.466, 0.5, 256))
colors_pos = plt.cm.RdBu_r(np.linspace(0.5, 1, 256))
all_colors = np.vstack((colors_neg,colors_pos))
orig = colors.LinearSegmentedColormap.from_list('orig',
    all_colors)
divnorm = colors.DivergingNorm(vmin=-1, vcenter=0, vmax=14)

im3 = axes[2].contourf(ta_ann.lat, ta_ann.plev/100, ta_ann,
                       cmap=orig,norm=divnorm,levels=np.arange(0,14,1))
cb3 = plt.colorbar(im3,ax=axes[2])
cb3.ax.tick_params(labelsize=8)
cb3.set_label('K',fontsize=8)
cb3.set_ticks(np.arange(0,14,1))

axes[0].tick_params(labelsize=8)
axes[0].set_title('(a) First 3 months',fontsize=8)
axes[1].tick_params(labelsize=8)
axes[1].set_title('(b) First year',fontsize=8)
axes[2].set_title('(c) Last 30 years',fontsize=8)
axes[2].tick_params(labelsize=8)
axes[0].set_xticks(np.arange(0,91,15))
axes[1].set_xticks(np.arange(0,91,15))
axes[2].set_xticks(np.arange(0,91,15))
axes[0].set_ylabel('Pressure (hPa)',fontsize=8)
axes[0].set_xlabel(r'Latitude (°N)',fontsize=8)
axes[1].set_xlabel('Latitude (°N)',fontsize=8)
axes[2].set_xlabel('Latitude (°N)',fontsize=8)

plt.tight_layout(pad=0)
# plt.show()
plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_3_060920.pdf')

<Figure size 532.8x216 with 6 Axes>

In [20]:
ta_mon.min().min()

<xarray.DataArray '__xarray_dataarray_variable__' ()>
array(-0.13503838)